In [151]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score,recall_score
from sklearn.impute import SimpleImputer

In [152]:
data = pd.read_csv("heart.csv")

Смотрим на наш датасет

In [153]:
data.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [154]:
categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

In [155]:
X_train, X_test, y_train, y_test = train_test_split(data,data['HeartDisease'], test_size=0.2, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [156]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [157]:
transform = list()

for cat_col in categorical_features:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    transform.append((cat_col, cat_transformer))
    
for cont_col in numerical_features:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    transform.append((cont_col, cont_transformer))

In [158]:
feats = FeatureUnion(transform)
feature_processing = Pipeline([('feats', feats)])

In [160]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier()),
])

pipeline.fit(X_train, y_train)

CPU times: user 368 ms, sys: 23.1 ms, total: 391 ms
Wall time: 570 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('RestingECG',
                                                 Pipeline(steps=[('selector',
    

In [161]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Sex',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='Sex')),
                                                  ('ohe',
                                                   OHEEncoder(key='Sex'))])),
                                 ('ChestPainType',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='ChestPainType')),
                                                  ('ohe',
                                                   OHEEncoder(key='ChestPainType'))])),
                                 ('RestingECG',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='RestingECG')),
                                                  ('ohe',
                                                   

In [162]:
with open("GB_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)